In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/weather-dataset/weather_dataset.csv


In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=77320fd73f8b352fcfbe372a0f8175ee28a4863c91e1157e3bd55fa6d734b67c
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassificationModel
import os

# Initialize Spark Session
spark = SparkSession.builder.appName('weather_prediction').getOrCreate()

# Load the dataset
file_path = '/kaggle/input/weather-dataset/weather_dataset.csv'
df = spark.read.csv(file_path, header=True, inferSchema=True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/14 11:22:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Selecting relevant columns and dropping rows with missing values
# Chọn ra các cột dữ liệu liên quan và loại bỏ những hàng có giá trị thiếu.
selected_columns = ['tempC', 'windspeedKmph', 'humidity', 'pressure', 'cloudcover', 'precipMM', 'weatherDesc']
df = df.select(selected_columns).na.drop()

In [5]:
# Indexing the label column
indexer = StringIndexer(inputCol='weatherDesc', outputCol='label')
indexer_model = indexer.fit(df)
df = indexer_model.transform(df)

In [6]:
df.show()

+-----+-------------+--------+--------+----------+--------+--------------------+-----+
|tempC|windspeedKmph|humidity|pressure|cloudcover|precipMM|         weatherDesc|label|
+-----+-------------+--------+--------+----------+--------+--------------------+-----+
|   25|            8|      97|    1007|        73|     0.2|Patchy rain possible|  1.0|
|   25|            8|      97|    1006|        45|     0.0|       Partly cloudy|  0.0|
|   25|            8|      97|    1006|        53|     0.0|       Partly cloudy|  0.0|
|   25|            8|      97|    1005|        61|     0.0|              Cloudy|  2.0|
|   25|            9|      96|    1005|        60|     0.0|              Cloudy|  2.0|
|   25|           11|      95|    1006|        59|     0.3|Patchy rain possible|  1.0|
|   25|           13|      94|    1006|        58|     0.1|Patchy rain possible|  1.0|
|   26|           12|      92|    1007|        51|     0.1|Patchy rain possible|  1.0|
|   27|           10|      90|    1007|    

In [7]:
# lưu lại StringIndexer
indexer_model.write().overwrite().save("/kaggle/working/StringIndexer")

In [8]:
df.select('weatherDesc', 'label').distinct().show()


+--------------------+-----+
|         weatherDesc|label|
+--------------------+-----+
| Heavy rain at times|  8.0|
|              Cloudy|  2.0|
|            Overcast|  3.0|
|               Sunny|  4.0|
|Moderate or heavy...|  6.0|
|       Partly cloudy|  0.0|
|Moderate rain at ...|  7.0|
|Patchy rain possible|  1.0|
|               Clear|  5.0|
|Patchy light drizzle| 15.0|
|                Mist| 10.0|
|   Patchy light rain| 19.0|
|Torrential rain s...| 17.0|
|Thundery outbreak...| 20.0|
|       Light drizzle| 11.0|
|          Light rain| 13.0|
|                 Fog| 12.0|
|       Moderate rain| 14.0|
|Patchy light rain...| 16.0|
|          Heavy rain| 18.0|
+--------------------+-----+
only showing top 20 rows



In [9]:
# Assembling the features
# Tạo vector đặc trưng (features) từ các cột dữ liệu như nhiệt độ, tốc độ gió, độ ẩm, áp suất, mây che phủ và lượng mưa bằng VectorAssembler.
assembler = VectorAssembler(inputCols=['tempC', 'windspeedKmph', 'humidity', 'pressure', 'cloudcover', 'precipMM'], outputCol='features')
df = assembler.transform(df)

In [10]:
df.show()

+-----+-------------+--------+--------+----------+--------+--------------------+-----+--------------------+
|tempC|windspeedKmph|humidity|pressure|cloudcover|precipMM|         weatherDesc|label|            features|
+-----+-------------+--------+--------+----------+--------+--------------------+-----+--------------------+
|   25|            8|      97|    1007|        73|     0.2|Patchy rain possible|  1.0|[25.0,8.0,97.0,10...|
|   25|            8|      97|    1006|        45|     0.0|       Partly cloudy|  0.0|[25.0,8.0,97.0,10...|
|   25|            8|      97|    1006|        53|     0.0|       Partly cloudy|  0.0|[25.0,8.0,97.0,10...|
|   25|            8|      97|    1005|        61|     0.0|              Cloudy|  2.0|[25.0,8.0,97.0,10...|
|   25|            9|      96|    1005|        60|     0.0|              Cloudy|  2.0|[25.0,9.0,96.0,10...|
|   25|           11|      95|    1006|        59|     0.3|Patchy rain possible|  1.0|[25.0,11.0,95.0,1...|
|   25|           13|      9

In [11]:
# Split the data into training and test sets
# Chia bộ dữ liệu thành hai phần: dữ liệu huấn luyện và dữ liệu kiểm thử.
train_data, test_data = df.randomSplit([0.8, 0.2])

In [12]:
# Initialize classifiers
# Khởi tạo hai mô hình phân loại: Random Forest và Logistic Regression.
rf = RandomForestClassifier(labelCol='label', featuresCol='features')
lr = LogisticRegression(labelCol='label', featuresCol='features')

In [13]:
# Initialize pipeline
# Xây dựng pipeline cho mỗi mô hình để huấn luyện trên dữ liệu.
pipeline_rf = Pipeline(stages=[rf])
pipeline_lr = Pipeline(stages=[lr])

In [14]:
# Fit the models
# Huấn luyện 2 mô hình
model_rf = pipeline_rf.fit(train_data)
model_lr = pipeline_lr.fit(train_data)

24/01/14 11:22:43 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/01/14 11:22:47 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.5
24/01/14 11:22:48 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.5
24/01/14 11:22:48 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.25
24/01/14 11:22:48 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.125
24/01/14 11:22:48 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.0625
24/01/14 11:22:49 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.0703125
24/01/14 11:22:49 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search zoom failed
24/01/14 11:22:53 ERROR Stron

In [15]:
# Save the Random Forest model
model_rf.save("/kaggle/working/random_forest_model1")

In [16]:
# Save the Logistic Regression model
model_lr.save("/kaggle/working/logistic_regression_model1")

In [17]:
# Make predictions
# Dự đoán trên một tập test
predictions_rf = model_rf.transform(test_data)
predictions_lr = model_lr.transform(test_data)

In [18]:
# Initialize evaluator
# Khởi tạo bộ đánh giá
# Sử dụng để đánh giá độ chính xác của mô hình phân loại đa lớp bằng cách so sánh các nhãn dự đoán với các nhãn thực tế trong tập dữ liệu
evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')

In [19]:
# Evaluate the models
# Đánh giá độ chính xác của hai mô hình
accuracy_rf = evaluator.evaluate(predictions_rf)
accuracy_lr = evaluator.evaluate(predictions_lr)

In [20]:
# Print the results
print('Random Forest Accuracy:', accuracy_rf)
print('Logistic Regression Accuracy:', accuracy_lr)

Random Forest Accuracy: 0.9152930177631255
Logistic Regression Accuracy: 0.8941101215371746


In [21]:
# Calculate precision, recall, f1-score
# Tính toán các chỉ số precision, recall, và f1-score cho hai mô hình
evaluator.setMetricName('weightedPrecision')
precision_rf = evaluator.evaluate(predictions_rf)
precision_lr = evaluator.evaluate(predictions_lr)

evaluator.setMetricName('weightedRecall')
recall_rf = evaluator.evaluate(predictions_rf)
recall_lr = evaluator.evaluate(predictions_lr)

evaluator.setMetricName('f1')
f1_score_rf = evaluator.evaluate(predictions_rf)
f1_score_lr = evaluator.evaluate(predictions_lr)

In [22]:
# Print the evaluation metrics
print('Random Forest Precision:', precision_rf)
print('Random Forest Recall:', recall_rf)
print('Random Forest F1-Score:', f1_score_rf)
print('Logistic Regression Precision:', precision_lr)
print('Logistic Regression Recall:', recall_lr)
print('Logistic Regression F1-Score:', f1_score_lr)

Random Forest Precision: 0.9144902079131594
Random Forest Recall: 0.9152930177631255
Random Forest F1-Score: 0.9130043587831037
Logistic Regression Precision: 0.8881620856527626
Logistic Regression Recall: 0.8941101215371747
Logistic Regression F1-Score: 0.8900699752138951


In [23]:
# Stop the Spark session
spark.stop()